# Week 3 Task3 Clustering the Neighborhoods of Toronto


In [1]:
#Import libraries
import numpy as np
import pandas as pd
import requests

#to handle json files
import json
!conda install -c conda-forge geopy --yes
from pandas.io.json import json_normalize

#Matplotlib libraries for plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

#k-means for clustering
from sklearn.cluster import KMeans

#map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported')


Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


foli

## **Step 1.** Download data set and transform it to data frame

In [15]:
#download data set from previous task
post_loc = pd.read_csv('PostalCodes2.csv')
post_loc

,PostalCode,Borough,Neighborhood,Postcode,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,M1B,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,M1C,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,M1K,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,M1L,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,M1M,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,M1N,43.692657,-79.264848


## **Step 2.** Building the map of Toronto neighborhoods

In [7]:
# use geopy library to fetch coordinates of Torornto
from geopy.geocoders import Nominatim
address = 'Toronto, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Location coordinates of Torornto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Location coordinates of Torornto are 43.6534817, -79.3839347.


In [10]:
#build the map of Torornto
map_toronto = folium.Map(location =[ latitude, longitude], zoom_start = 10)

#add markers to the map
for lat, lng, borough, neighborhood in zip(post_loc['Latitude'], post_loc['Longitude'], 
                                          post_loc['Borough'], post_loc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat,lng], radius = 5, popup = label, color = 'blue', 
                        fill = True, fill_color = '#3186cc', fill_opacity = 0.7, parse_html = False).add_to(map_toronto)
map_toronto

Let's reduce the circle of neighborhoods being explored.
There are 5 trendy neighborhoods in Toronto:
*Old town and Distillery District*
*The Entertainment District*
*Yorkville + the Annex*
*Chinatown + Kensington Market + AGO District*
*Queen West + King West + Liberty Village*
All of them located in Downtown. Let's create a new data frame to explore the Downtown

In [16]:
#create new data frame for Downtown
downtown_data = post_loc[post_loc['Borough'] == 'Downtown Toronto'].reset_index(drop = True)
downtown_data.head()

,PostalCode,Borough,Neighborhood,Postcode,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,M4W,43.679563,-79.377529
1,M4X,Downtown Toronto,St. James Town / Cabbagetown,M4X,43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
3,M5A,Downtown Toronto,Regent Park / Harbourfront,M5A,43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937


In [18]:
#fetch location data for Downtown
address = 'Downtown Toronto, Toronto, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Location coordinates of Toronto Downtown are {}, {}.'.format(latitude, longitude))


Location coordinates of Toronto Downtown are 43.6563221, -79.3809161.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [24]:
#create Downtown map
map_downtown = folium.Map(location =[ latitude, longitude], zoom_start = 12)

#add markers to the map
for lat, lng, borough, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], 
                                          downtown_data['Borough'], downtown_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat,lng], radius = 5, popup = label, color = 'blue', 
                        fill = True, fill_color = '#3186cc', fill_opacity = 0.7, parse_html = False).add_to(map_downtown)
map_downtown

## **Step3.** Get data from FourSquare API to explore neighborhoods

In [25]:
#define FourSquare Credentials
CLIENT_ID = 'FZSTTYYU51YBXTPQEJR5IS0YWX4BBRT0DIPLXE0TSXAYQFAH'
CLIENT_SECRET = 'MVPRTHA1ZS43AWARFNM3MBY5JTOVFCK3SJTRDRLLBMWBK2CF'
VERSION = '20200501'

Now let' get top 150 venues that are in Downtown Town within 1 kilometer radius

In [80]:
#the function retrieves venues given the names and coordinates and stores it into data frame
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    downtown_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    downtown_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(downtown_venues)

In [81]:
#retrive al the venues at given address
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'], latitudes = downtown_data['Latitude'],
                                 longitudes = downtown_data['Longitude'])

Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Queen's Park / Ontario Provincial Government


In [82]:
print(downtown_venues.shape)
downtown_venues.head()

(1671, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Summerhill Market,43.686265,-79.375458,Grocery Store
1,Rosedale,43.679563,-79.377529,Black Camel,43.677016,-79.389367,BBQ Joint
2,Rosedale,43.679563,-79.377529,Toronto Lawn Tennis Club,43.680667,-79.388559,Athletics & Sports
3,Rosedale,43.679563,-79.377529,Pie Squared,43.672143,-79.377856,Pie Shop
4,Rosedale,43.679563,-79.377529,Tinuno,43.671281,-79.374920,Filipino Restaurant


## ** Step 4.** Analyse each neighborhood

In [83]:
#group venues in each neighborhood
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
Commerce Court / Victoria Hotel,100,100,100,100,100,100
First Canadian Place / Underground city,100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
Harbourfront East / Union Station / Toronto Islands,100,100,100,100,100,100


In [85]:
#one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix = "", prefix_sep = "")

#add neighborhood column to the data frame
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood']

fixed_columns = [downtown_onehot.columns[-1]]+list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]
downtown_onehot.head()

,Yoga Studio,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Theme Restaurant,Track,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
#new data frame size
downtown_onehot.shape

(1671, 194)

In [87]:
#grouping rows by neighborhood and by taking mean frequency of occurence of each category
downtown_gr = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_gr

,Neighborhood,Yoga Studio,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Theme Restaurant,Track,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.00,0.0000,0.0000,0.01,0.00,0.00,0.02,0.00,0.00,...,0.00,0.0000,0.000000,0.01,0.00,0.01,0.00,0.00,0.00,0.00
1,CN Tower / King and Spadina / Railway Lands / ...,0.00,0.0625,0.0625,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0625,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Central Bay Street,0.02,0.0000,0.0000,0.01,0.00,0.00,0.02,0.01,0.02,...,0.00,0.0000,0.000000,0.00,0.01,0.02,0.00,0.00,0.00,0.00
3,Christie,0.00,0.0000,0.0000,0.01,0.00,0.00,0.01,0.00,0.00,...,0.00,0.0000,0.000000,0.00,0.00,0.02,0.00,0.01,0.01,0.00
4,Church and Wellesley,0.02,0.0000,0.0000,0.01,0.00,0.00,0.01,0.00,0.01,...,0.01,0.0000,0.000000,0.00,0.00,0.00,0.01,0.00,0.00,0.01
5,Commerce Court / Victoria Hotel,0.00,0.0000,0.0000,0.02,0.00,0.00,0.02,0.00,0.00,...,0.00,0.0000,0.000000,0.01,0.00,0.02,0.00,0.00,0.00,0.00
6,First Canadian Place / Underground city,0.00,0.0000,0.0000,0.02,0.00,0.00,0.01,0.00,0.00,...,0.00,0.0000,0.000000,0.01,0.00,0.02,0.00,0.00,0.00,0.00
7,"Garden District, Ryerson",0.01,0.0000,0.0000,0.01,0.00,0.00,0.00,0.01,0.01,...,0.00,0.0000,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.00
8,Harbourfront East / Union Station / Toronto Is...,0.01,0.0000,0.0000,0.00,0.00,0.02,0.01,0.00,0.00,...,0.00,0.0100,0.000000,0.01,0.00,0.02,0.00,0.00,0.00,0.00
9,Kensington Market / Chinatown / Grange Park,0.03,0.0000,0.0000,0.00,0.00,0.00,0.03,0.00,0.02,...,0.00,0.0000,0.000000,0.00,0.01,0.04,0.00,0.02,0.01,0.00


In [88]:
#print each neighborhood along the top5 most common venues
num_top_venues = 5

for hood in downtown_gr['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_gr[downtown_gr['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.12
1                 Café  0.07
2                Hotel  0.06
3  Japanese Restaurant  0.04
4           Restaurant  0.04


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
              venue  freq
0       Coffee Shop  0.12
1              Café  0.12
2   Harbor / Marina  0.12
3  Sculpture Garden  0.06
4    Airport Lounge  0.06


----Central Bay Street----
              venue  freq
0       Coffee Shop  0.10
1              Café  0.05
2  Sushi Restaurant  0.03
3    Clothing Store  0.03
4              Park  0.03


----Christie----
               venue  freq
0  Korean Restaurant  0.12
1               Café  0.07
2        Coffee Shop  0.06
3      Grocery Store  0.05
4       Cocktail Bar  0.03


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.12
1     Sushi Restaurant  0.05
2  Japanese Restaurant  0.04
3             

In [91]:
#create pandas data frame and display top 10 venues in each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_gr['Neighborhood']

for ind in np.arange(downtown_gr.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_gr.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Park,Bakery,Grocery Store,Gym,Liquor Store
1,CN Tower / King and Spadina / Railway Lands / ...,Café,Coffee Shop,Harbor / Marina,Park,Sushi Restaurant,Dog Run,Scenic Lookout,Sculpture Garden,Garden,Track
2,Central Bay Street,Coffee Shop,Café,Gastropub,Sushi Restaurant,Park,Clothing Store,Hotel,Plaza,Middle Eastern Restaurant,Mexican Restaurant
3,Christie,Korean Restaurant,Café,Coffee Shop,Grocery Store,Cocktail Bar,Mexican Restaurant,Ice Cream Shop,Japanese Restaurant,Pizza Place,Park
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Park,Japanese Restaurant,Diner,Caribbean Restaurant,Ramen Restaurant,Men's Store,Restaurant,Italian Restaurant


## **Step 5.** Cluster Neighborhoods

We will run *k*-means to cluster neighborhood into 5 clusters

In [94]:
kclusters = 5

downtown_gr_clustering = downtown_gr.drop('Neighborhood', 1)

#run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_gr_clustering)

#check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head(3) 

ValueError: cannot insert Cluster Labels, already exists

In [96]:
#create the cluster map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], 
                                  downtown_merged['Neighborhood'],downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## **Step6** Examine clusters

In [97]:
#Cluster 1
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,-79.375418,0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Gastropub,Italian Restaurant,Theater,Hotel,Art Gallery,Gym
6,Downtown Toronto,-79.373306,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Park,Bakery,Grocery Store,Gym,Liquor Store
8,Downtown Toronto,-79.384568,0,Café,Coffee Shop,Hotel,Theater,Restaurant,Tea Room,Japanese Restaurant,Plaza,Pizza Place,Furniture / Home Store
9,Downtown Toronto,-79.381752,0,Coffee Shop,Café,Hotel,Park,Japanese Restaurant,Gym,Theater,Brewery,Plaza,Pizza Place
10,Downtown Toronto,-79.381576,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Concert Hall,Seafood Restaurant,Theater,Vegetarian / Vegan Restaurant,Monument / Landmark
11,Downtown Toronto,-79.379817,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Seafood Restaurant,Restaurant,Concert Hall,Bookstore,Gym,Plaza
15,Downtown Toronto,-79.374846,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Seafood Restaurant,Gastropub,Park,Bakery,Gym
16,Downtown Toronto,-79.382280,0,Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Seafood Restaurant,Concert Hall,Theater,Plaza,Park


In [98]:
#Cluster 2
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,-79.367675,1,Park,Restaurant,Diner,Japanese Restaurant,Café,Gastropub,Filipino Restaurant,Pub,Pool,Coffee Shop
2,Downtown Toronto,-79.383160,1,Coffee Shop,Sushi Restaurant,Park,Japanese Restaurant,Diner,Caribbean Restaurant,Ramen Restaurant,Men's Store,Restaurant,Italian Restaurant
3,Downtown Toronto,-79.360636,1,Coffee Shop,Pub,Diner,Café,Park,Theater,Italian Restaurant,Restaurant,Bakery,Breakfast Spot
4,Downtown Toronto,-79.378937,1,Coffee Shop,Gastropub,Japanese Restaurant,Café,Restaurant,Diner,Theater,Hotel,Middle Eastern Restaurant,Creperie
7,Downtown Toronto,-79.387383,1,Coffee Shop,Café,Gastropub,Sushi Restaurant,Park,Clothing Store,Hotel,Plaza,Middle Eastern Restaurant,Mexican Restaurant
18,Downtown Toronto,-79.389494,1,Coffee Shop,Sushi Restaurant,Park,Ramen Restaurant,Gastropub,Café,Clothing Store,Middle Eastern Restaurant,Restaurant,Japanese Restaurant


In [99]:
#Cluster 3
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,-79.39442,2,Café,Coffee Shop,Harbor / Marina,Park,Sushi Restaurant,Dog Run,Scenic Lookout,Sculpture Garden,Garden,Track


In [100]:
#Cluster 4
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.377529,3,Coffee Shop,Park,Grocery Store,Breakfast Spot,Athletics & Sports,Metro Station,Candy Store,Sandwich Place,Bistro,Filipino Restaurant


In [101]:
#Cluster 5
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,-79.400049,4,Café,Bakery,Bar,Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Bookstore,Beer Bar,Arts & Crafts Store
13,Downtown Toronto,-79.400049,4,Café,Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Yoga Studio,Art Gallery,Mexican Restaurant,Bakery,Dessert Shop,Record Shop
17,Downtown Toronto,-79.422564,4,Korean Restaurant,Café,Coffee Shop,Grocery Store,Cocktail Bar,Mexican Restaurant,Ice Cream Shop,Japanese Restaurant,Pizza Place,Park


## Looks like Cluster 1 is good for city break - plenty hotels and places to visit and eat